In [3]:
import pandas as pd

df = pd.read_csv('creditcard.csv')

In [4]:
# Since most of our data has already been scaled we should scale the columns that are left to scale (Amount and Time)
from sklearn.preprocessing import StandardScaler, RobustScaler

# RobustScaler is less prone to outliers.

std_scaler = StandardScaler()
rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)

scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

# Amount and Time are Scaled!

df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [108]:
from sklearn.model_selection import train_test_split

#X=data.iloc[:, :-1]
#y=data.iloc[:, -1]

#X = data.loc[:, 'V1' : 'V28']
#y = data.loc[:, 'Class']

X = data.drop('Class', axis=1)
y = data['Class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)


#원본 데이터의 비율과 비슷한 비율로 나누어 졌는지 확인
print(y_train.value_counts() / y_train.count() *100 )
print(y_test.value_counts() / y_test.count() *100 )

0    99.82532
1     0.17468
Name: Class, dtype: float64
0    99.834978
1     0.165022
Name: Class, dtype: float64


In [109]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

n_inputs = X_train.shape[1]
n_outputs = 2
n_latent = 50

inputs = tf.keras.layers.Input(shape=(n_inputs, ))
x = tf.keras.layers.Dense(100, activation='tanh')(inputs)
latent = tf.keras.layers.Dense(n_latent, activation='tanh')(x)

# Encoder
encoder = tf.keras.models.Model(inputs, latent, name='encoder')
encoder.summary()

latent_inputs = tf.keras.layers.Input(shape=(n_latent, ))
x = tf.keras.layers.Dense(100, activation='tanh')(latent_inputs)
outputs = tf.keras.layers.Dense(n_inputs, activation='sigmoid')(x)

# Decoder
decoder = tf.keras.models.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

# 정상 데이터 만을 학습
x_train_norm = X_train[y_train == 0]

autoencoder = tf.keras.models.Model(inputs, decoder(encoder(inputs)))
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x_train_norm, x_train_norm, epochs=15, batch_size = 100, validation_data=(X_test, X_test))


Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
dense_104 (Dense)            (None, 100)               3100      
_________________________________________________________________
dense_105 (Dense)            (None, 50)                5050      
Total params: 8,150
Trainable params: 8,150
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        [(None, 50)]              0         
_________________________________________________________________
dense_106 (Dense)            (None, 100)               5100      
________________________________________

KeyboardInterrupt: 

In [ ]:
encoded = encoder.predict(X_train)

'''
classifier = tf.keras.Sequential([
    layers.Dense(32, input_dim=n_latent, activation='tanh'),
    layers.Dense(16, activation='relu'),
    layers.Dense(n_outputs, activation ='softmax')
])
classifier.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.summary()
classifier.fit(encoded, y_train, batch_size=100, epochs=10)
'''

In [ ]:
'''
y_pred = classifier.predict(encoder.predict(X_test))

print(accuracy_score(y_test, y_pred))


print('Accuracy Score: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print('Recall Score: {:.2f}'.format(recall_score(y_test, y_pred)))
print('Precision Score: {:.2f}'.format(precision_score(y_test, y_pred)))
print('F1 Score: {:.2f}'.format(f1_score(y_test, y_pred)))
print('ROC AUC Score: {:.2f}'.format(roc_auc_score(y_test, y_pred)))
'''

In [ ]:
'''
from sklearn.metrics import classification_report
pred_y = classifier.predict(encoder.predict(X_test)).argmax(axis=1)
y = y_test

print(classification_report(y, pred_y))
'''

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

classifiers = {
    "LogisiticRegression": LogisticRegression(max_iter=1000),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "RandomForest Classifier": RandomForestClassifier()
}

In [93]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, roc_auc_score

for key, classifier in classifiers.items():
    classifier.fit(encoded, y_train)
    training_score = cross_val_score(classifier, X_train,y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")
   
    y_pred = classifier.predict(encoder.predict(X_test))
    print('Accuracy Score: {:.2f}'.format(accuracy_score(y_test, y_pred)))
    print('Recall Score: {:.2f}'.format(recall_score(y_test, y_pred)))
    print('Precision Score: {:.2f}'.format(precision_score(y_test, y_pred)))
    print('F1 Score: {:.2f}'.format(f1_score(y_test, y_pred)))
    print('ROC AUC Score: {:.2f}'.format(roc_auc_score(y_test, y_pred)))
    

Classifiers:  LogisticRegression Has a training score of 100.0 % accuracy score
Accuracy Score: 1.00
Recall Score: 0.79
Precision Score: 0.91
F1 Score: 0.85
ROC AUC Score: 0.89
Classifiers:  KNeighborsClassifier Has a training score of 100.0 % accuracy score
Accuracy Score: 1.00
Recall Score: 0.80
Precision Score: 0.96
F1 Score: 0.87
ROC AUC Score: 0.90
Classifiers:  SVC Has a training score of 100.0 % accuracy score
Accuracy Score: 1.00
Recall Score: 0.82
Precision Score: 0.92
F1 Score: 0.87
ROC AUC Score: 0.91
Classifiers:  RandomForestClassifier Has a training score of 100.0 % accuracy score
Accuracy Score: 1.00
Recall Score: 0.80
Precision Score: 0.97
F1 Score: 0.88
ROC AUC Score: 0.90
